# Pass or fail

Phân tích kết quả thi của học sinh, xác định trạng thái đậu/rớt, và xuất danh sách kết quả chi tiết vào một file CSV.

In [ ]:
import pandas as pd
import requests
from io import StringIO
from datetime import datetime, timedelta
import os


cleaned_data_api = 'https://andyanh.id.vn/index.php/s/psPTAMbDrzzMnWk/download'
tinh_api = 'https://andyanh.id.vn/index.php/s/zbHTAjksBekNB4M/download'

#tải giữa liệu từ api và lưu vào cache, đặt thời gian hiệu lực cho file cache
def fetch_csv_from_api(api_url):

    
    cache_file = 'cleaned_data_cache.csv' if 'AQrka' in api_url else 'tinh_cache.csv'
    cache_timeout = timedelta(hours=24)
# Kiểm tra cache, nếu có tồn tại thì lấy thời gian chỉnh sửa lần cuối của file và đọc file
    if os.path.exists(cache_file):
        modified_time = datetime.fromtimestamp(os.path.getmtime(cache_file))
        if datetime.now() - modified_time < cache_timeout:
            print(f"Đang tải dữ liệu từ cache {cache_file}...")  
            return pd.read_csv(cache_file)
        
#gửi yêu cầu get đến api để tải dữ liệu, (StringIO(response.text)) : đọc dữ liệu tư chuỗi phần hồi api

    print(f"Đang tải dữ liệu từ API {api_url}...")
    response = requests.get(api_url)
    if response.status_code == 200:
        df = pd.read_csv(StringIO(response.text))
        df.to_csv(cache_file, index=False) #lưu dữ liệu mới vào file cache để sử dụng lần sau
        return df
    else:
        raise Exception(f"Failed to fetch data: {response.status_code}") #Nếu không tải được (HTTP lỗi), thông báo lỗi.

# Tải dữ liệu từ API
try: # lưu dữ liệu từ hai api vào data frame
    cleaned_data_df = fetch_csv_from_api(cleaned_data_api)
    tinh_df = fetch_csv_from_api(tinh_api)
    print("Đã tải dữ liệu thành công từ API")
except Exception as e:
    print(f"Lỗi khi tải dữ liệu từ API: {e}")
    print("Không thể tải dữ liệu. Vui lòng kiểm tra kết nối internet và thử lại.")
    exit()

# Merge data based on 'MaTinh'
merged_df = pd.merge(cleaned_data_df, tinh_df, on='MaTinh', how='left') # kết hợp hai bảng dữ liệu theo cột chung Matinh
# how='left': Giữ tất cả dữ liệu từ bảng bên trái (cleaned_data_df) và bổ sung thông tin từ bảng bên phải (tinh_df).

# Calculate the total score across all subjects for each student
merged_df['Total_Score'] = merged_df[['Toan', 'Van', 'Ly', 'Sinh', 'Ngoai ngu', 'Hoa', 'Lich su', 'Dia ly', 'GDCD']].sum(axis=1, skipna=True)
#sum(axis=1): Tính tổng điểm từng hàng (học sinh) qua các môn.
#skipna=True: Bỏ qua giá trị NaN khi tính tổng.


# Count the number of subjects each student has taken
merged_df['Subjects_Taken'] = merged_df[['Toan', 'Van', 'Ly', 'Sinh', 'Ngoai ngu', 'Hoa', 'Lich su', 'Dia ly', 'GDCD']].notnull().sum(axis=1)
#notnull(): Xác định các môn không bị NaN (học sinh có điểm).
#sum(axis=1): Đếm số môn đã thi (không phải NaN) cho từng học sinh.

# Label students with 3 or fewer subjects as "Thí sinh thi lại"
merged_df['Ket_Qua'] = merged_df['Subjects_Taken'].apply(lambda x: "Thí sinh thi lại" if x <= 3 else None)
#sắp xếp lại kết, nếu số môn đã thi <=3, gắn nhẵn "Thí sinh thi lại" còn lại để trống (none)

# Assuming a passing score threshold (e.g., 15)
passing_score = 15
merged_df['Ket_Qua'] = merged_df.apply(
    lambda row: "Đậu" if row['Total_Score'] >= passing_score and row['Ket_Qua'] is None else  #Nếu tổng điểm ≥ passing_score (15) và chưa có kết quả, gắn "Đậu".
                ("Rớt" if row['Ket_Qua'] is None else row['Ket_Qua']), #Nếu chưa có kết quả và không đủ điểm, gắn "Rớt",Giữ nguyên kết quả trước đó (nếu có).
    axis=1 #Áp dụng logic cho từng hàng.
)
# xác định đậu rớt dựa trên tổng điểm, 


# Add 'Nam' column based on a custom logic or pattern in 'SBD' or another field
# Example: Assume year data is extracted from some pattern in 'SBD'
#Xác định năm thi dựa trên SBD
merged_df['Nam'] = merged_df['SBD'].astype(str).apply(
    lambda x: 2018 if "18" in x else (2019 if "19" in x else None)  # Adjust the condition as needed
)#Chuyển SBD thành chuỗi, xác định năm thi (2018 hoặc 2019) dựa trên chuỗi con trong SBD.

# Filter for students from the years 2018 and 2019
merged_df = merged_df[merged_df['Nam'].isin([2018, 2019])] #Giữ lại chỉ các học sinh thi năm 2018 hoặc 2019.
#Lọc dữ liệu theo năm

# Sort by year (2018 first, 2019 second)
merged_df = merged_df.sort_values(by='Nam') #Sắp xếp kết quả theo năm thi (2018 trước, 2019 sau).

# Select relevant columns for the final output, Chọn các cột cần thiết
result_df = merged_df[['SBD', 'Total_Score', 'Subjects_Taken', 'Ket_Qua', 'TenTinh', 'Nam']].round(2)
#Chọn cột: Giữ lại các thông tin quan trọng như SBD, tổng điểm, số môn, kết quả, tên tỉnh, năm thi.

# Export to CSV
result_df.to_csv('Pass_Fail_List_2018_2019.csv', index=False)
#to_csv(): Xuất kết quả ra file CSV với tên Pass_Fail_List_2018_2019.csv.
